In [1]:
import pyopencl as cl
from pyopencl.elementwise import ElementwiseKernel
import pyopencl.array as cl_array
import pyopencl.reduction as cl_reduction

import numpy as np

from elementwise import *
from c_code import *

In [2]:
class GPUOPERATOR:
    """
    GPU operators with return values still stored on GPU. 
    """

    def __init__(self, context, queue):
        self.context = context
        self.queue = queue
        self.program = cl.Program(self.context, c_code).build()

    def add(self, A, B):
        """A, B are assumed to be on the device"""
        height, width = A.shape 
        heightB, widthB = B.shape    
        assert height == heightB and width == widthB, "Arrays have different shapes."
        out = cl_array.zeros_like(A)

        self.program.ew_add(self.queue, (width, height), None, 
                            A.data, B.data, np.int32(width), out.data).wait()
        return out
        
    def sub(self, A, B):
        """A, B are assumed to be on the device"""
        height, width = A.shape 
        heightB, widthB = B.shape    
        assert height == heightB and width == widthB, "Arrays have different shapes."
        out = cl_array.zeros_like(A)

        self.program.ew_sub(self.queue, (width, height), None, 
                            A.data, B.data, np.int32(width), out.data).wait()
        return out

    def div(self, A, B):
        """A, B are assumed to be on the device"""
        height, width = A.shape 
        heightB, widthB = B.shape    
        assert height == heightB and width == widthB, "Arrays have different shapes."
        out = cl_array.zeros_like(A)

        self.program.ew_div(self.queue, (width, height), None, 
                            A.data, B.data, np.int32(width), out.data).wait()
        return out

    def mul(self, A, B):
        """A, B are assumed to be on the device"""
        height, width = A.shape 
        heightB, widthB = B.shape    
        assert height == heightB and width == widthB, "Arrays have different shapes."
        out = cl_array.zeros_like(A)

        self.program.ew_mul(self.queue, (width, height), None, 
                            A.data, B.data, np.int32(width), out.data).wait()
        return out

    def greater(self, A, B):
        """A, B are assumed to be on the device"""
        height, width = A.shape 
        heightB, widthB = B.shape    
        assert height == heightB and width == widthB, "Arrays have different shapes."
        out = cl_array.zeros_like(A)

        self.program.ew_greater(self.queue, (width, height), None, 
                               A.data, B.data, np.int32(width), out.data).wait()
        return out

    def equal(self, A, B):
        """A, B are assumed to be on the device"""
        height, width = A.shape 
        heightB, widthB = B.shape    
        assert height == heightB and width == widthB, "Arrays have different shapes."
        out = cl_array.zeros_like(A)

        self.program.ew_equal(self.queue, (width, height), None, 
                               A.data, B.data, np.int32(width), out.data).wait()
        return out

    def matmul(self, A, B):
        """A, B are assumed to be on the device"""
        heightA, widthA = A.shape
        heightB, widthB = B.shape
        assert widthA == heightB, "Cannot do matrix multiplication."
        C = np.empty((heightA, widthB), dtype=np.float32)
        out = cl_array.to_device(self.queue, C)

        BLOCK_SIZE = 16
        local_size = (BLOCK_SIZE, BLOCK_SIZE)

        self.program.matrixmultiply2dlocal(self.queue, (heightA, widthB), local_size, 
                    np.int32(heightA), np.int32(widthA), np.int32(heightB), np.int32(widthB), A.data, B.data, out.data).wait()
        return out
    
    def transpose(self, A):
        """A is assumed to be on the device"""
        global_size = A.shape
        local_size = (16, 16)

        width, height = A.shape

        A_transpose = cl_array.zeros_like(A)
        a_local = cl.LocalMemory(4 * 16 * (16 + 1))
        
        self.program.transpose(self.queue, global_size, local_size,
                               A_transpose.data, A.data, np.int32(width), np.int32(height), a_local).wait()
        return A_transpose

    def sum(self, A, axis=None):
        """A is assumed to be on the device"""
        heightA, widthA = A.shape

        if axis == None:
            rk = cl_reduction.ReductionKernel(self.context, np.float32, neutral="0", reduce_expr="a+b", map_expr="a[i]",
                            arguments="__global const float *a")
            output_sum = rk(A)
            return output_sum        
    
        else:
            if axis == 1:
                A = self.transpose(A).copy()
            
            r = np.empty(A.shape[1]).astype(np.float32)
            r_gpu = cl_array.to_device(self.queue, r)
            self.program.reduce(self.queue, (A.shape[0]*A.shape[1], ), None, 
                                A.data, r_gpu.data, np.int32(A.shape[1]), np.int32(A.shape[0])).wait()
            return r_gpu
    
    def exp(self, A):
        """A is assumed to be on the device"""
        out = cl_array.zeros_like(A)
        programme = ElementwiseKernel(self.context,"float *a_gpu, float *out",
                                          "out[i] = exp(a_gpu[i])",
                                          "exponential")
        
        programme(A, out)
        return out

    def log(self, A):
        """A is assumed to be on the device"""
        out = cl_array.zeros_like(A)
        programme = ElementwiseKernel(self.context,"float *a_gpu, float *out",
                                          "out[i] = log(a_gpu[i])",
                                          "logarithm")
        programme(A, out)
        return out

    def pow(self, A, b):
        """A is assumed to be on the device"""
        out = cl_array.zeros_like(A)
        programme = ElementwiseKernel(self.context,"float *a_gpu, float b, float *out",
                                          "out[i] = pow(a_gpu[i],b)",
                                          "power")
        programme(A, b, out)
        return out
    
    def sqrt(self, A):
        """A is assumed to be on the device"""
        out = cl_array.zeros_like(A)
        programme = ElementwiseKernel(self.context,"float *a_gpu, float *out",
                                          "out[i] = sqrt(a_gpu[i])",
                                          "sqrt")
        programme(A, out)
        return out

    def abs(self, A):
        """A is assumed to be on the device"""
        out = cl_array.zeros_like(A)
        programme = ElementwiseKernel(self.context,"float *a_gpu, float *out",
                                          "out[i] = fabs(a_gpu[i])",
                                          "abs")
        programme(A, out)
        return out

    def relu(self, A):
        """A is assumed to be on the device"""
        out = cl_array.zeros_like(A)
        programme = ElementwiseKernel(self.context,"float *a_gpu, float *out",
                                          "out[i] = 0.5* (a_gpu[i] + fabs(a_gpu[i]))",
                                          "relu")
        programme(A, out)
        return out
    
    def tanh(self, A):
        """A is assumed to be on the device"""
        out = cl_array.zeros_like(A)
        programme = ElementwiseKernel(self.context,"float *a_gpu, float *out",
                                          "out[i] = tanh(a_gpu[i])",
                                          "tanh")
        programme(A, out)
        return out

    def sigmoid(self, A):
        """A is assumed to be on the device"""
        out = cl_array.zeros_like(A)
        programme = ElementwiseKernel(self.context,
                                    "float *x, float *out",
                                    "out[i] = SIGMOID(x[i])",
                                    "sigmoid",
                                    preamble='#define SIGMOID(x) x > 0 ? 1.0/(1.0 + exp(-x)) : exp(x) / (exp(x) + 1.0)'
                                    )
        programme(A, out)
        return out
    
    def sign(self, A):
        """A is assumed to be on the device"""
        height, width = A.shape    
        out = cl_array.zeros_like(A)

        self.program.ew_sign(self.queue, (width, height), None, 
                               A.data, np.int32(width), out.data).wait()
        return out 
    
    def swish(self, A):
        """A is assumed to be on the device"""
        out = cl_array.zeros_like(A)
        programme = ElementwiseKernel(self.context,
                                    "float *x, float *out",
                                    "out[i] = SWISH(x[i])",
                                    "swish",
                                    preamble='#define SWISH(x) x > 0 ? x/(1.0 + exp(-x)) : x*exp(x) / (exp(x) + 1.0)'
                                    )
        programme(A, out)
        return out

In [3]:
platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)

operator = GPUOPERATOR(context, queue)

In [4]:
# Some tests of the GPU operators
m, n= 2**8, 2**8
A = np.random.rand(m, n).astype(np.float32)
B = np.random.rand(m, n).astype(np.float32)

C = cl_array.to_device(queue, A)
D = cl_array.to_device(queue, B)

add_AB = operator.add(C, D)
matmal_AB = operator.matmul(C, D)
power_3 = operator.pow(C, 3)

sum0 = operator.sum(C, axis=0) 

np.testing.assert_almost_equal(add_AB.get(), A+B, decimal=3)
np.testing.assert_almost_equal(matmal_AB.get(), A@B, decimal=2)
np.testing.assert_almost_equal(power_3.get(), A**3, decimal=2)
np.testing.assert_almost_equal(sum0.get(), np.sum(A, axis=0), decimal=2)

In [5]:
add_AB 

cl.Array([[1.6126754 , 1.0972321 , 0.4661155 , ..., 0.49850494, 1.5177472 ,
        1.4337666 ],
       [0.9034496 , 1.6443253 , 1.2025478 , ..., 0.7884326 , 0.70104986,
        0.9244702 ],
       [1.1405876 , 0.78726274, 1.2581811 , ..., 0.8236594 , 1.7612596 ,
        0.7834784 ],
       ...,
       [1.316417  , 1.5965381 , 0.38418406, ..., 0.6757586 , 1.0045685 ,
        1.2331691 ],
       [1.8708758 , 0.38737434, 0.6372671 , ..., 1.5209737 , 1.398317  ,
        1.1475902 ],
       [1.2556428 , 0.753708  , 0.5914285 , ..., 1.3551574 , 1.0851675 ,
        1.3129367 ]], dtype=float32)

In [6]:
A+B

array([[1.6126754 , 1.0972321 , 0.4661155 , ..., 0.49850494, 1.5177472 ,
        1.4337666 ],
       [0.9034496 , 1.6443253 , 1.2025478 , ..., 0.7884326 , 0.70104986,
        0.9244702 ],
       [1.1405876 , 0.78726274, 1.2581811 , ..., 0.8236594 , 1.7612596 ,
        0.7834784 ],
       ...,
       [1.316417  , 1.5965381 , 0.38418406, ..., 0.6757586 , 1.0045685 ,
        1.2331691 ],
       [1.8708758 , 0.38737434, 0.6372671 , ..., 1.5209737 , 1.398317  ,
        1.1475902 ],
       [1.2556428 , 0.753708  , 0.5914285 , ..., 1.3551574 , 1.0851675 ,
        1.3129367 ]], dtype=float32)

In [9]:
a = np.array([[1,2],[3,4]])

True

In [11]:
a.astype(bool)

array([[ True,  True],
       [ True,  True]])